In [2]:
from transformers import AutoTokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/root/group-shared/jrc/base_models/Qwen3-14B")

In [4]:
prompt = tokenizer.apply_chat_template(
    [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": f"Question: question"
        }
    ],
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)
print(prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Question: question<|im_end|>
<|im_start|>assistant
<think>

</think>


